In [12]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API Key:")

Enter your OpenAI API Key:··········


In [13]:
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass('Enter your LangSmith API key: ')

Enter your LangSmith API key: ··········


In [14]:
from uuid import uuid4

unique_id = uuid4().hex[0:8]

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"Managing_Agent_Prompt_Size_{unique_id}"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"

In [ ]:
pip install langchain langchain-openai langchain-community

In [16]:
from operator import itemgetter

from langchain.agents import AgentExecutor, load_tools
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain_community.tools import WikipediaQueryRun, DuckDuckGoSearchRun
from langchain_community.utilities import WikipediaAPIWrapper, DuckDuckGoSearchAPIWrapper
from langchain_core.prompt_values import ChatPromptValue
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI

# Managing Prompt Length

- Agents dynamically use tools to gather information, adding the results to their prompts.

- This can lead to large prompts that exceed the model's context window.

- LCEL allows for custom functions to manage prompt size within an agent.

- Example: An agent that searches Wikipedia for information.

- It's important to trim the prompt by keeping only necessary information and removing the rest.

In [ ]:
pip install wikipedia duckduckgo-search

In [18]:
wiki = WikipediaQueryRun(
    api_wrapper=WikipediaAPIWrapper(top_k_results=10, doc_content_chars_max=10_000)
)

ddg_search = search = DuckDuckGoSearchRun(
    api_wrapper=DuckDuckGoSearchAPIWrapper(region="us-en", time="d", max_results=5)
)

tools = [ddg_search, wiki ]

In [19]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", """You are the world's greatest research assistant. You know exactly
        where and what to search for given a query. You've been a research assistant to
        people like Yann LeCun, Geoffry Hinton, and Francois Chollet."""),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

llm = ChatOpenAI(model="gpt-3.5-turbo")

In [20]:
agent = (
    {
        "input": itemgetter("input"),
        "agent_scratchpad": lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm.bind_functions(tools)
    | OpenAIFunctionsAgentOutputParser()
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [21]:
agent_executor.invoke(
    {
        "input": """Why does deep learning work? Is there some inherent property
        of a model's architecture that makes it capable of learning? Or is it more
        about the data it is trained on?  Are there any theorems or hypothesis that you can find that
        support this? Is there something special about how humans generate data?
        What philosophical implications does this have about why deep learning works.
        How will people in the future talk about what deep learning is?"""
    }
)



> Entering new AgentExecutor chain...
Deep learning works due to a combination of factors, including the architecture of the model, the data it is trained on, and the optimization algorithms used during training. Here are some key points to consider:

1. **Architecture of the Model**: Deep learning models, particularly neural networks, are capable of learning complex patterns and representations from data due to their hierarchical structure. Each layer in a neural network extracts features at different levels of abstraction, allowing the model to learn intricate patterns in the data.

2. **Data**: The quality and quantity of data used to train a deep learning model play a crucial role in its performance. Deep learning models require large amounts of labeled data to generalize well and make accurate predictions. The diversity and representativeness of the data also impact the model's ability to learn meaningful patterns.

3. **Optimization Algorithms**: Deep learning models are traine

{'input': "Why does deep learning work? Is there some inherent property\n        of a model's architecture that makes it capable of learning? Or is it more\n        about the data it is trained on?  Are there any theorems or hypothesis that you can find that\n        support this? Is there something special about how humans generate data?\n        What philosophical implications does this have about why deep learning works.\n        How will people in the future talk about what deep learning is?",
 'output': "Deep learning works due to a combination of factors, including the architecture of the model, the data it is trained on, and the optimization algorithms used during training. Here are some key points to consider:\n\n1. **Architecture of the Model**: Deep learning models, particularly neural networks, are capable of learning complex patterns and representations from data due to their hierarchical structure. Each layer in a neural network extracts features at different levels of abs

In [22]:
def condense_prompt(prompt: ChatPromptValue, max_tokens: int = 4_000) -> ChatPromptValue:
    """
    Condenses the input prompt to ensure the total number of tokens does not exceed a specified limit.

    This function processes a ChatPromptValue object's messages to reduce the total token count to
    within a specified limit. It progressively removes messages from the beginning of the AI function-related
    messages until the total token count is under the specified limit, while always preserving the first
    two messages for essential context or instructions.

    Parameters:
    - prompt (ChatPromptValue): The input ChatPromptValue object containing a sequence of messages.
    - max_tokens (int, optional): The maximum number of tokens allowed for the condensed prompt. Defaults to 4,000.

    Returns:
    - ChatPromptValue: A new ChatPromptValue object with the condensed sequence of messages, ensuring
      the total token count is within the specified limit.

    Note:
    - This function is useful for scenarios where the prompt for an AI model exceeds the maximum token limit,
      allowing for the inclusion of necessary context while staying within token constraints.
    """
    messages = prompt.to_messages()
    num_tokens = llm.get_num_tokens_from_messages(messages)
    ai_function_messages = messages[2:]

    while num_tokens > max_tokens:
        ai_function_messages = ai_function_messages[2:]
        num_tokens = llm.get_num_tokens_from_messages(
            messages[:2] + ai_function_messages
        )

    messages = messages[:2] + ai_function_messages
    return ChatPromptValue(messages=messages)


In [23]:
agent = (
    {
        "input": itemgetter("input"),
        "agent_scratchpad": lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | condense_prompt
    | llm.bind_functions(tools)
    | OpenAIFunctionsAgentOutputParser()
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent_executor.invoke(
    {
        "input": """Why does deep learning work? Is there some inherent property
        of a model's architecture that makes it capable of learning? Or is it more
        about the data it is trained on?  Are there any theorems or hypothesis that you can find that
        support this? Is there something special about how humans generate data?
        What philosophical implications does this have about why deep learning works.
        How will people in the future talk about what deep learning is?"""
    }
)



> Entering new AgentExecutor chain...
Deep learning works due to a combination of factors, including the architecture of the model, the data it is trained on, and the optimization process. Here are some insights into the questions you posed:

1. **Architecture of the Model**: The architecture of a deep learning model, such as neural networks with multiple layers, enables it to learn complex patterns and representations from the data. The hierarchical structure of deep learning models allows them to capture intricate relationships in the data, making them powerful for tasks like image recognition, natural language processing, and more.

2. **Data it is Trained On**: The quality and quantity of data used to train a deep learning model play a crucial role in its performance. Deep learning models require large amounts of labeled data to generalize well and make accurate predictions. The diversity and richness of the training data contribute to the model's ability to learn meaningful feat

{'input': "Why does deep learning work? Is there some inherent property\n        of a model's architecture that makes it capable of learning? Or is it more\n        about the data it is trained on?  Are there any theorems or hypothesis that you can find that\n        support this? Is there something special about how humans generate data?\n        What philosophical implications does this have about why deep learning works.\n        How will people in the future talk about what deep learning is?",
 'output': "Deep learning works due to a combination of factors, including the architecture of the model, the data it is trained on, and the optimization process. Here are some insights into the questions you posed:\n\n1. **Architecture of the Model**: The architecture of a deep learning model, such as neural networks with multiple layers, enables it to learn complex patterns and representations from the data. The hierarchical structure of deep learning models allows them to capture intricate